In [250]:
import os
import csv
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import nltk 
import operator
import math
import scipy

In [427]:
global_POS = []
f = open('/home/rahmtin/Documents/Arxiv-Feynmann/Arxiv/Part Of Speech/POS.txt')
for line in f:
    global_POS.append(line[0:2])

global_POS = list(set(global_POS))
relevant_features = ['VB', 'NN', 'RB', 'JJ', 'DT']


In [408]:
class Paper:
    def __init__(self, title):
         self.title = title
         self.authors = []
         self.experiences = []
         self.year = ''
         self.pos = []
         self.features = dict()
         self.tag = ""
         
    def to_string(self):
        return [self.title, self.year, self.authors, self.features]
    

f = open('/home/rahmtin/Documents/Arxiv-Feynmann/Arxiv/Part Of Speech/TitlesAndAuthorsAndMonth4.txt')
papers = []
paper = Paper('test')
counter = 0;
global_features = [':', '?' , 'MATH']
tag = ""
author_papers = dict()

for line in f:
    try:
        text = nltk.word_tokenize(line)
    except UnicodeDecodeError:
        for line2 in f:
            if len(nltk.word_tokenize(line2)) == 0:
                break
        continue

    if len(text) == 0:
        papers.append(paper)
        counter = counter + 1
#        if counter >= 50:
#            break
        continue

    if text[0] == 'Tag':
        tag = text[2]
        
    if text[0] == 'Title':
        title = (' '.join(text[2:])).lower().decode('utf-8', 'ignore').encode('utf-8', 'ignore')
        number = 0
        tmp = ''
        for x in title:
            if x == '$':
                if number == 1:
                    tmp = tmp + " MATH "
                number = 1 - number
                continue
            if number > 0:
                continue
            tmp = tmp + x
        paper = Paper(' '.join(tmp.split()))
        paper.pos = nltk.pos_tag(nltk.word_tokenize(paper.title))
        if len(paper.pos) > 0:
            paper.pos[0] = (paper.pos[0][0], paper.pos[0][1][:2])
        paper.tag = tag
        for feature in global_features: 
            paper.features[feature] = 0
            if feature in paper.title:
                paper.features[feature] = 1
            
    
    if text[0] == 'Year':
        if text[2][0] == '9':
            text[2] = '19' + text[2]
        else:
            text[2] = '20' + text[2]
        paper.year = text[2]
    if text[0] == 'Author':
        author = ' '.join(text[2:]).lower()
        paper.authors.append(' '.join(author.split()))
    
papers.sort(key=operator.attrgetter('year'))
author_experience = dict()
author_papers = dict()
for index, paper in enumerate(papers):
    for author in paper.authors:
        if author not in author_experience:
            author_experience[author] = 0
            author_papers[author] = []
        author_papers[author].append(index)
        paper.experiences.append(author_experience[author])
        author_experience[author] = author_experience[author] + 1

#    print "---->", paper.authors, paper.year
#    print paper.experiences

In [409]:
import sys
def find_scores(paper_list):
    if(len(paper_list) < min_for_min_threshold):
        sys.exit()
    ret = dict()
    for feature in global_features:
        ret[feature] = 0
    for feature in relevant_features:
        ret[feature] = 0
    for index in paper_list:
        paper = papers[index]
        for feature in global_features:
            ret[feature] = ret[feature] + int(paper.features[feature])
        for feature in relevant_features:
            if len(paper.pos) > 0:
                ret[feature] = ret[feature] + int(paper.pos[0][1] == feature)
    return ret

In [460]:
import random
#author_papers = dict()
older_bigger_fights = []
younger_bigger_fights = []
fight_count = 0
author_pairs = set()
regression_scores = dict()
regression_label = dict()
denom = dict()
num = dict()
ratio = dict()
first_pos_point = []

min_for_min_threshold = 1
max_for_min_threshold = 2
final_experience_threshold_for_younger = 10

for feature in global_features + relevant_features:
    denom[feature] = dict()
    num[feature] = dict()
    ratio[feature] = dict()
    regression_scores[feature] = []
    regression_label[feature] = []
    one_sided_regression_denom[feature] = dict()
    one_sided_regression_num[feature] = dict()
    
for index, paper in enumerate(papers):
    if len(paper.authors) == 2 and len(paper.pos) > 0:
        if tuple(paper.authors) not in author_pairs:
            older_index = younger_index = 0
            if paper.experiences[0] < paper.experiences[1]: 
                older_index = 1
            else:
                younger_index = 1
            
            if(min(paper.experiences) >= min_for_min_threshold and min(paper.experiences) <= max_for_min_threshold and
              paper.experiences[older_index] / paper.experiences[younger_index] > 2 and 
              paper.experiences[older_index] - paper.experiences[younger_index] > 10 and
              author_experience[paper.authors[younger_index]] > final_experience_threshold_for_younger):
                # we found a fight
                fight_count += 1
                older_papers = author_papers[paper.authors[older_index]]
                younger_papers = author_papers[paper.authors[younger_index]]
                temp_older_papers = older_papers
                older_papers = list(set(older_papers) - set(younger_papers))
                younger_papers = list(set(younger_papers) - set(temp_older_papers))
                older_scores = find_scores(older_papers)
                younger_scores = find_scores(younger_papers)
                for feature in global_features + relevant_features:
                    younger_score = younger_scores[feature]
                    older_score = older_scores[feature]
                    if (younger_score, older_score) not in denom[feature]:
                        denom[feature][(younger_score, older_score)] = 0
                        num[feature][(younger_score, older_score)] = 0
                    denom[feature][(younger_score, older_score)] = denom[feature][(younger_score, older_score)] + 1
                    regression_scores[feature].append([younger_score, older_score])
                    younger_prob = float(younger_score) / len(younger_papers) 
                    older_prob = float(older_score) / len(older_papers)
                    my_tuple = (younger_prob, min(paper.experiences),
                                older_prob, max(paper.experiences))
                    
                    if my_tuple not in one_sided_regression_denom[feature]: 
                        one_sided_regression_denom[feature][my_tuple] = 0
                        one_sided_regression_num[feature][my_tuple] = 0
                    one_sided_regression_denom[feature][my_tuple] += 1    
                    
                    if feature in global_features:
                        regression_label[feature].append(paper.features[feature])
                        if paper.features[feature] == 0:
                            my_tuple = my_tuple + (0,)
                            continue
                        num[feature][(younger_score, older_score)] = num[feature][(younger_score, older_score)] + 1
                        one_sided_regression_num[feature][my_tuple] += 1
                        my_tuple = my_tuple + (1,)
                   
                    if feature in relevant_features:
                        regression_label[feature].append(int(paper.pos[0][1] == feature))
                        num[feature][(younger_score, older_score)] = (num[feature][(younger_score, older_score)] +
                                                                      int(paper.pos[0][1] == feature))
                        one_sided_regression_num[feature][my_tuple] += int(paper.pos[0][1] == feature)
                        my_tuple = my_tuple + (int(paper.pos[0][1] == feature),)
                    if older_prob > younger_prob:
                        older_bigger_fights.append(my_tuple)
                    elif younger_prob > older_prob:
                        younger_bigger_fights.append(my_tuple)
                  
                                
    for author1 in paper.authors:
        for author2 in paper.authors:
            author_pairs.add((author1, author2))
            author_pairs.add((author2, author1))
#        if author1 not in author_papers:
#            author_papers[author1] = []
#        author1_papers = author_papers[author1]
#        while len(author1_papers) >= min_for_min_threshold:
#            author1_papers.pop(0)
#        author1_papers.append(index)
#        author_papers[author1] = author1_papers

print fight_count
for feature in global_features + relevant_features:
    copy1 = dict()
    copy2 = dict()
    for my_tuple in denom[feature]:
        denom[feature][my_tuple] = max(1, denom[feature][my_tuple])
        ratio[feature][my_tuple] =  (num[feature][my_tuple] / float(denom[feature][my_tuple]))
        (i,j) = my_tuple
        if ((j,i) not in denom[feature]):
            copy1[(j, i)] = 1
            copy2[(j, i)] = 0
    denom[feature].update(copy1)
    num[feature].update(copy2)
    ratio[feature].update(copy2)

1291


In [471]:
#match younger_bigger_fights with older_bigger_fights
import copy
print len(older_bigger_fights), len(younger_bigger_fights)
matches = 0
older_fights = copy.copy(older_bigger_fights)
younger_fights = copy.copy(younger_bigger_fights)
younger_won_duel = 0
older_won_duel = 0

for older_fight in older_fights: 
    for younger_fight in younger_fights:
        if ((abs(younger_fight[0] - older_fight[2]) < 1e-2) and (abs(younger_fight[2] - older_fight[0]) < 1e-2) and
            abs(younger_fight[4] - older_fight[4]) == 1): # the probs are pretty close and different outcomes
            matches += 1
     #       print younger_fight[4], older_fight[4]
            if younger_fight[4] == 1:
                younger_won_duel += 1
            else:
                older_won_duel += 1
            younger_fights.remove(younger_fight)
            break
print matches, younger_won_duel, older_won_duel, scipy.stats.binom_test(older_won_duel, matches)

3352 2779
1077 448 629 3.85611303246e-08


In [474]:
for feature in relevant_features + global_features:
    for (i,j) in num[feature]:
        if i < j: # i is younger, j is older
        #(younger less usage, older more usage) - (older less usage, younger more usage)
                    
            p_value = scipy.stats.binom_test(num[feature][(j,i)], denom[feature][(j,i)], ratio[feature][(i,j)], alternative='two-sided')
               
            mean1 = ratio[feature][(i,j)]
            var1 = mean1 * (1 - mean1)
                #print mean1, (1 - mean1), var1
            mean2 = ratio[feature][(j,i)]
            var2 = mean2 * (1 - mean2)
                #print mean2, (1 - mean2), var2
            sample_numbers = denom[feature][(i,j)] + denom[feature][(j,i)]
            CI = 1.96  * (var1 + var2) / (math.sqrt(denom[feature][(i,j)]) + math.sqrt(denom[feature][(j,i)]))
    
        #print (feature, i, j,
        #       str(ratio[feature][(i,j)] - ratio[feature][(j,i)]) + " +/- " + str(CI), p_value,
        #       num[feature][(i,j)], denom[feature][(i,j)],
        #       num[feature][(j,i)], denom[feature][(j,i)] )


In [418]:
## One sided/Both sided tuple experiment: regression
from random import randrange
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import preprocessing

relevant_features = ['VB', 'NN', 'RB', 'JJ', 'DT']
f = open('/home/rahmtin/Documents/Arxiv-Feynmann/Arxiv/DominantName/Regression.txt', 'w')
def do_regression(data):
    for feature in relevant_features + global_features:
        all_features = []
        all_labels = []
        one_sided_regression_features = [[], []]
        one_sided_regression_labels = [[], []]
        for key in data[feature]:
            all_features.append(key)
            all_labels.append(data[feature][key])
    
        all_features = preprocessing.scale(all_features)
        for index in range(len(all_features)):
            random_index = randrange(0,2)
            one_sided_regression_features[random_index].append(all_features[index])
            one_sided_regression_labels[random_index].append(all_labels[index])
    
#    print one_sided_regression_features[0][0], one_sided_regression_features[1][0]
#    print one_sided_regression_labels[0][0], one_sided_regression_labels[1][0]
#    break
        clf = linear_model.LinearRegression() # first one is younger, second one is older
        clf.fit(one_sided_regression_features[0], one_sided_regression_labels[0])
        prediction = clf.predict(one_sided_regression_features[1])
        five_decimals = ["%.5f" % v for v in clf.coef_]
        five_decimals_intercept = ["%.5f" % clf.intercept_]
        print >>f, feature, ' Coefficients: ', five_decimals, five_decimals_intercept
        print >>f, "R^2 score: ", r2_score(one_sided_regression_labels[1], prediction)
        

one_sided_num_compact = dict()
one_sided_denom_compact = dict()
young_sided_regression_ratio = dict()
old_sided_regression_ratio = dict()
both_sided_regression_ratio = dict()
relevant_features = ['VB', 'NN', 'RB', 'JJ', 'DT']

for feature in global_features + relevant_features:
    young_sided_regression_ratio[feature] = dict()
    old_sided_regression_ratio[feature] = dict()
    one_sided_num_compact[feature] = dict()
    one_sided_denom_compact[feature] = dict()
    both_sided_regression_ratio[feature] = dict()

for feature in global_features + relevant_features:
    for my_tuple in one_sided_regression_num[feature]:
        # (young_prob, young_exp, old_prob, old_exp)
        both_sided_regression_ratio[feature][my_tuple] = (one_sided_regression_num[feature][my_tuple] / 
                                                   max(one_sided_regression_denom[feature][my_tuple], 1))
        compact_tuple = (my_tuple[0], my_tuple[1], my_tuple[3])
        if compact_tuple not in one_sided_num_compact[feature]:
            one_sided_num_compact[feature][compact_tuple] = 0
            one_sided_denom_compact[feature][compact_tuple] = 0
        one_sided_num_compact[feature][compact_tuple] += one_sided_regression_num[feature][my_tuple]
        one_sided_denom_compact[feature][compact_tuple] += one_sided_regression_denom[feature][my_tuple]
    for compact_tuple in one_sided_denom_compact[feature]:
        young_sided_regression_ratio[feature][compact_tuple] = (one_sided_num_compact[feature][compact_tuple] / 
                                                    max(one_sided_denom_compact[feature][compact_tuple], 1))

for feature in global_features + relevant_features:
    one_sided_num_compact[feature] = dict()
    one_sided_denom_compact[feature] = dict()
    for my_tuple in one_sided_regression_num[feature]:
        # (young_prob, young_exp, old_prob, old_exp)
        compact_tuple = (my_tuple[2], my_tuple[3], my_tuple[1])
        if compact_tuple not in one_sided_num_compact[feature]:
            one_sided_num_compact[feature][compact_tuple] = 0
            one_sided_denom_compact[feature][compact_tuple] = 0
        one_sided_num_compact[feature][compact_tuple] += one_sided_regression_num[feature][my_tuple]
        one_sided_denom_compact[feature][compact_tuple] += one_sided_regression_denom[feature][my_tuple]
    for compact_tuple in one_sided_denom_compact[feature]:
        old_sided_regression_ratio[feature][compact_tuple] = (one_sided_num_compact[feature][compact_tuple] / 
                                                    max(one_sided_denom_compact[feature][compact_tuple], 1))
                                                 

print >>f, "OLD: (old_prob, old_experience, young_experience), intercept"
do_regression(old_sided_regression_ratio)
print >>f, ""
print >>f, "YOUNG: (young_prob, young_experience, old_experience), intercept"                                               
do_regression(young_sided_regression_ratio)
print >>f, ""
print >>f, "ALL: (young_prob, young_experience, old_prob, old_experience), intercept"                      
do_regression(both_sided_regression_ratio)


In [485]:
## One sided/Both sided tuple experiment: logistic regression
from random import randrange
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

relevant_features = ['VB', 'NN', 'RB', 'JJ', 'DT']
f = open('/home/rahmtin/Documents/Arxiv-Feynmann/Arxiv/DominantName/LogisticRegression.txt', 'w')
def do_logistic_regression(data, labels):
    for feature in relevant_features + global_features:
        all_features = []
        all_labels = []
        one_sided_regression_features = [[], []]
        one_sided_regression_labels = [[], []]
        for index, key in enumerate(data[feature]):
            all_features.append(data[feature][index])
            all_labels.append(labels[feature][index])
    
        all_features = preprocessing.scale(all_features)
        for index in range(len(all_features)):
            random_index = randrange(0,2)
            one_sided_regression_features[random_index].append(all_features[index])
            one_sided_regression_labels[random_index].append(all_labels[index])
    
#    print one_sided_regression_features[0][0], one_sided_regression_features[1][0]
#    print one_sided_regression_labels[0][0], one_sided_regression_labels[1][0]
#    break
        logistic = linear_model.LogisticRegression(C=1e5)
        logistic.fit(one_sided_regression_features[0], one_sided_regression_labels[0])
        LogisticRegression(C=100000.0, class_weight=None, dual=False,
            fit_intercept=True, intercept_scaling=1, max_iter=100,
            multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
            solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
        
        prediction = logistic.predict(one_sided_regression_features[1])
        five_decimals = ["%.5f" % v for v in logistic.coef_[0]]
        five_decimals_intercept = ["%.5f" % logistic.intercept_]
        print >>f, feature, ' Coefficients: ', five_decimals, five_decimals_intercept
        print >>f, "Accuracy: ",  accuracy_score(one_sided_regression_labels[1], prediction)
        


young_features = dict()
old_features = dict()
both_features = dict()
young_labels = dict()
old_labels = dict()
both_labels = dict()
relevant_features = ['VB', 'NN', 'RB', 'JJ', 'DT']

for feature in global_features + relevant_features:
    young_features[feature] = []
    old_features[feature] = []
    both_features[feature] = []
    young_labels[feature] = []
    old_labels[feature] = []
    both_labels[feature] = []
    

for feature in global_features + relevant_features:
    for my_tuple in one_sided_regression_num[feature]:
        # (young_prob, young_exp, old_prob, old_exp)
        young_compact_tuple = (my_tuple[0], my_tuple[1], my_tuple[3])
        old_compact_tuple = (my_tuple[2], my_tuple[3], my_tuple[1])
        for i in range(one_sided_regression_denom[feature][my_tuple]):
            label = 0
            if i < one_sided_regression_num[feature][my_tuple]:
                label = 1      
            both_features[feature].append(my_tuple)
            both_labels[feature].append(label)
            young_features[feature].append(young_compact_tuple)
            young_labels[feature].append(label)
            old_features[feature].append(old_compact_tuple)
            old_labels[feature].append(label)
            
                                                 

print >>f, "OLD: (old_prob, old_experience, young_experience), intercept"
do_logistic_regression(old_features, old_labels)
print >>f, ""
print >>f, "YOUNG: (young_prob, young_experience, old_experience), intercept"                                               
do_logistic_regression(young_features, young_labels)
print >>f, ""
print >>f, "ALL: (young_prob, young_experience, old_prob, old_experience), intercept"                      
do_logistic_regression(both_features, both_labels)


In [354]:
# from sklearn import linear_model
clf = linear_model.LinearRegression() # first one is younger, second one is older
#print regression_scores[':']
clf.fit(regression_scores[':'], regression_label[':'])
print(': Coefficients: \n', clf.coef_)
clf.fit(regression_scores['?'], regression_label['?'])
print('? Coefficients: \n', clf.coef_)
clf.fit(regression_scores['MATH'], regression_label['MATH'])
print('MATH Coefficients: \n', clf.coef_)

(': Coefficients: \n', array([ 0.04366242,  0.00843984]))
('? Coefficients: \n', array([ 0.04736798,  0.01052285]))
('MATH Coefficients: \n', array([ 0.11733052,  0.00641327]))


In [249]:

older_wins_count = 0                                                                           
younger_wins_count = 0
for x in first_pos_point:
    if x[0] < x[1]:
        older_wins_count += 1
    elif x[1] < x[0]:
        younger_wins_count += 1

print "younger, older wins: ", younger_wins, older_wins

younger, older wins:  781 761


In [469]:
?r2_score
